In [3]:
import os
import shutil
import importlib
import copy
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import torch
import torchaudio

import phaselocknet_model
import util

importlib.reload(phaselocknet_model)
importlib.reload(util)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


device(type='cuda', index=0)

In [8]:
regex_filenames = "/media/marmoset/data/code_package/phaselock/localization/mp_spectral_cues_stim.hdf5" 
sr = 50000 if "localization" in regex_filenames else 20000
keys = util.get_hdf5_dataset_key_list(regex_filenames)
dataset = util.HDF5Dataset(regex_filenames, keys=keys[:-1])

In [26]:
dataset[8010]

{'dbspl': 58.925613,
 'foreground_azimuth': 180,
 'foreground_elevation': 30,
 'foreground_wav': b'/om2/scratch/Fri/francl/aud_scenes_hebrank_wright/615_10.wav                                                                                                                                                                                                    ',
 'foreground_wav_clip_pos': array([    0, 88200]),
 'idx': 24,
 'index': 8010,
 'loc_idx': 255,
 'noise_high': 20000,
 'noise_low': 5800,
 'signal': array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], dtype=float32)}

In [9]:
# import numpy as np
# import pandas as pd
from tqdm import tqdm  # progress bar
# --- config ---
skip_like = {'signal'}     # avoid large arrays
outfile_csv = "stimuli_all_para.csv"         # output file name
# --------------

def to_scalar(x):
    # Convert numpy scalars / 0-d arrays to Python scalars
    if isinstance(x, (np.generic,)):
        return x.item()
    if isinstance(x, np.ndarray) and x.shape == ():
        return x.item()
    return x

# Determine which keys to keep (prefer dataset.keys attribute if present)
if hasattr(dataset, "keys") and isinstance(dataset.keys, (list, tuple)):
    candidate_keys = [k for k in dataset.keys if k not in skip_like]
else:
    probe = dataset[0]
    candidate_keys = [k for k in probe.keys() if k not in skip_like]

rows = []
# n = len(dataset)
for i in tqdm(range(len(dataset))):
# for i in range(n):
    sample = dataset[i]  # dict per sample
    row = {}
    for k in candidate_keys:
        v = sample[k]
        # coerce simple arrays/scalars to python scalars
        try:
            if isinstance(v, np.ndarray):
                if v.ndim == 0:
                    row[k] = v.item()
                elif v.size == 1:
                    row[k] = v.reshape(()).item()
                else:
                    # if accidental vector, store a short repr
                    row[k] = str(v.shape)
            else:
                row[k] = to_scalar(v)
        except Exception:
            row[k] = str(v)  # last resort

    rows.append(row)

df = pd.DataFrame(rows)

# Make sure SNR is numeric (keeps inf as np.inf; unparsable -> NaN)
if 'snr' in df.columns:
    df['snr'] = pd.to_numeric(df['snr'], errors='coerce')

# Optional: stable sort for readability
# for key in ['segment_id', 'index', 'foreground_index_brir']:
#     if key in df.columns:
#         df = df.sort_values(by=key, kind='mergesort')
#         break

# Save
# df.to_csv(outfile_csv, index=False)
# print(f"Saved {len(df):,} rows to {outfile_csv}")

# (Optional) also save Parquet (smaller, faster)
# df.to_parquet("room0_para.parquet", index=False)


100%|██████████████████████████████████████| 9800/9800 [00:13<00:00, 729.07it/s]


In [10]:
df

,dbspl,foreground_azimuth,foreground_elevation,foreground_wav,foreground_wav_clip_pos,idx,index,loc_idx,noise_high,noise_low
0,57.536018,0,0,b'/om2/scratch/Fri/francl/aud_scenes_hebrank_w...,"(2,)",10,0,0,20000,10000
1,57.391678,0,10,b'/om2/scratch/Fri/francl/aud_scenes_hebrank_w...,"(2,)",10,1,1,20000,10000
2,58.990337,180,30,b'/om2/scratch/Fri/francl/aud_scenes_hebrank_w...,"(2,)",10,2,255,20000,10000
3,57.678173,180,40,b'/om2/scratch/Fri/francl/aud_scenes_hebrank_w...,"(2,)",10,3,256,20000,10000
4,57.008320,180,50,b'/om2/scratch/Fri/francl/aud_scenes_hebrank_w...,"(2,)",10,4,257,20000,10000
...,...,...,...,...,...,...,...,...,...,...
9795,58.872318,0,50,b'/om2/scratch/Fri/francl/aud_scenes_hebrank_w...,"(2,)",19,9795,5,20000,10000
9796,55.959103,0,60,b'/om2/scratch/Fri/francl/aud_scenes_hebrank_w...,"(2,)",19,9796,6,20000,10000
9797,58.940128,180,0,b'/om2/scratch/Fri/francl/aud_scenes_hebrank_w...,"(2,)",19,9797,252,20000,10000
9798,58.285946,180,10,b'/om2/scratch/Fri/francl/aud_scenes_hebrank_w...,"(2,)",19,9798,253,20000,10000


In [14]:
df["foreground_azimuth"].value_counts()

foreground_azimuth
0      4900
180    4900
Name: count, dtype: int64

In [18]:
df["foreground_elevation"].value_counts()

foreground_elevation
0     1400
10    1400
30    1400
40    1400
50    1400
60    1400
20    1400
Name: count, dtype: int64

In [24]:
df["loc_idx"].value_counts()

loc_idx
0      700
1      700
255    700
256    700
257    700
258    700
2      700
3      700
4      700
5      700
6      700
252    700
253    700
254    700
Name: count, dtype: int64

In [1]:
df["noise_high"].value_counts()

NameError: name 'df' is not defined

In [20]:
df["noise_low"].value_counts()

noise_low
20       5600
10000     700
15300     700
13200     700
3800      700
5800      700
7500      700
Name: count, dtype: int64